In [1]:
import argparse
import time
import os


In [2]:
os.chdir(os.path.abspath(os.path.join('..')))

In [13]:
from src.alo import ALO
from src.alo import AssetStructure

In [4]:
alo = ALO()

In [5]:
# alo 시작 전 setting
alo.preset()

In [6]:
# pipeline list 를 가지고 옴
pipelines = list(alo.asset_source.keys())

In [7]:
# 외부 데이터 가져오기
from src.external import external_load_data, external_save_artifacts
external_load_data(pipelines[0], alo.external_path, alo.external_path_permission, alo.control['get_external_data'])

[2023-10-31 01:46:56,277][PROCESS][INFO]: You did not write any << s3_private_key_file >> in the config yaml file. When you wanna get data from s3 storage, 
                                 you have to write the s3_private_key_file path or set << ACCESS_KEY, SECRET_KEY >> in your os environment. 

[2023-10-31 01:46:56,279][PROCESS][INFO]:  Start loading external data. << /nas001/users/yoonji.suh/tcr_test_20231016/train_multiclass/ >>  
 << train_multiclass >> does not exist in << /home/wonjun.sung/0.repo/release/release-1.2/input/ >>. 
 & << get_external_data >> is set as << once >>. 



In [8]:
# 사용하는 pipeline의 package를 설치
# train = 0, infernence = 1을 선택해야 하고 둘다 설치 해야함
pipeline = pipelines[0]
alo.install_steps(pipeline, alo.control["get_asset_source"])

[2023-10-31 01:47:01,543][PROCESS][INFO]: Start setting-up << input >> asset @ << assets >> directory.
[2023-10-31 01:47:01,544][PROCESS][INFO]: Start renewing asset : /home/wonjun.sung/0.repo/release/release-1.2/assets/input
[2023-10-31 01:47:01,707][PROCESS][INFO]: /home/wonjun.sung/0.repo/release/release-1.2/assets/input successfully pulled.
[2023-10-31 01:47:01,708][PROCESS][INFO]: Start setting-up << preprocess >> asset @ << assets >> directory.
[2023-10-31 01:47:01,708][PROCESS][INFO]: Start renewing asset : /home/wonjun.sung/0.repo/release/release-1.2/assets/preprocess
[2023-10-31 01:47:01,855][PROCESS][INFO]: /home/wonjun.sung/0.repo/release/release-1.2/assets/preprocess successfully pulled.
[2023-10-31 01:47:01,856][PROCESS][INFO]: Start setting-up << sampling >> asset @ << assets >> directory.
[2023-10-31 01:47:01,856][PROCESS][INFO]: Start renewing asset : /home/wonjun.sung/0.repo/release/release-1.2/assets/sampling
[2023-10-31 01:47:01,985][PROCESS][INFO]: /home/wonjun.sung

In [19]:
# 진행할 step을 list형식으로 입력하기
step = 0
# 입력할 args를 변수로 저장
input_args = alo.user_parameters[pipeline][step]['args'][0]

In [ ]:
# 사용자가 args를 변경해서 사용 가능
# input_args[0]['input_path'] = 'test'

In [11]:
# alo 내부에서 확인하는 config를 셋팅
asset_config = alo.asset_source[pipeline]
asset_config[step]

{'step': 'input',
 'source': {'code': 'http://mod.lge.com/hub/smartdata/ml-framework/alov2-module/input.git',
  'branch': 'tabular_dev',
  'requirements': ['pandas==1.5.3']}}

In [20]:
# 초기 data structure 구성
envs, args, data, config = {}, input_args, {}, {}
asset_structure = AssetStructure(envs, args, data, config)

alo.set_proc_logger()

In [21]:

data_input, config_input = alo.process_asset_step(asset_config[step], step, pipeline, asset_structure)

[2023-10-31 10:52:38,935][USER][INFO][train_pipeline][input]: >> Load path : ['/home/wonjun.sung/0.repo/release/release-1.2//input/train_multiclass/']
[2023-10-31 10:52:38,939][USER][INFO][train_pipeline][input]: >> The file for batch data has been loaded. (File name: /home/wonjun.sung/0.repo/release/release-1.2//input/train_multiclass/iris.csv)
[2023-10-31 10:52:38,940][USER][INFO][train_pipeline][input]: ==================== Success loading dataframe ====================
[2023-10-31 10:52:38,941][USER][INFO][train_pipeline][input]: >> Drop columns from the input dataframe when set << auto >> mode or specified in the << drop_columns >> in config yaml. (dropped colums:[])
[2023-10-31 10:52:38,942][USER][INFO][train_pipeline][input]: >> Start processing ignore columns & drop columns: ['/home/wonjun.sung/0.repo/release/release-1.2//input/train_multiclass/iris.csv']


[2023-10-31 01:52:38,934][ASSET][INFO][train_pipeline][input]: 

============================= ASSET START =============================
- time (UTC)        : 2023-10-31 01:52:38
- current step      : input
- asset branch.     : tabular_dev
- alolib ver.       : 0
- alo ver.          : develop
- load envs. keys   : dict_keys(['project_home', 'pipeline', 'step', 'num_step', 'artifacts', 'alo_version', 'asset_branch', 'interface_mode', 'load_data', 'load_config', 'save_data', 'save_config', 'log_file_path'])
- load args. keys   : dict_keys(['input_path', 'x_columns', 'use_all_x', 'y_column', 'groupkey_columns', 'drop_columns', 'time_column', 'concat_dataframes', 'encoding'])
- load config. keys : dict_keys([])
- load data keys    : dict_keys([])


[2023-10-31 01:52:38,943][ASSET][INFO][train_pipeline][input]: 

============================= ASSET FINISH ===========================
- time (UTC)        : 2023-10-31 01:52:38
- current step      : input
- save config. keys : dict_keys(['data

AttributeError: 'NoneType' object has no attribute 'process_info'

In [ ]:
step = 1
input_args = alo.user_parameters[pipeline][step]['args']

In [ ]:
asset_config = alo.asset_source[pipeline]
asset_config[step]

In [ ]:
data_input, config_input = alo.process_asset_step(asset_config[step], step, pipeline, data_input, config_input, input_args)

In [ ]:
step = 2
input_args = alo.user_parameters[pipeline][step]['args']
input_args

In [ ]:
asset_config = alo.asset_source[pipeline]
asset_config[step]

In [ ]:
data_input, config_input = alo.process_asset_step(asset_config[step], step, pipeline, data_input, config_input, input_args)

In [ ]:
step = 3
input_args = alo.user_parameters[pipeline][step]['args']
input_args

In [ ]:
asset_config = alo.asset_source[pipeline]
asset_config[step]

In [ ]:
data_input, config_input = alo.process_asset_step(asset_config[step], step, pipeline, data_input, config_input, input_args)